In [2]:
!pip install kagglehub
!pip install nltk

In [1]:
try:
    nltk.download("punkt")
    nltk.download("punkt_tab")
    nltk.download("wordnet")
    nltk.download("stopwords")
    nltk.download("averaged_perceptron_tagger")
    nltk.download("averaged_perceptron_tagger_eng")
except:
    print("Downloading punkt package was failed")

import nltk
import string
import kagglehub
import pandas as pd
import numpy as np
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf

BATCH_SIZE = 64

import tensorflow as tf

print(
    "\n",
    50 * "=",
    "\nNum GPUs Available: ",
    len(tf.config.list_physical_devices("GPU")),
    "\n",
    50 * "=",
)

# Download latest version
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")
_ds = pd.read_csv(
    path + "/twitter_training.csv",
    header=None,
    names=["Category", "Page", "Target", "Text"],
)
print(_ds.shape)
_ds.head()

/home/ai/Desktop/DeepLearning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-24 17:28:50.134545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758722330.148188  165517 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758722330.152446  165517 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758722330.163097  165517 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more tha


Num GPUs Available:  1 
(74682, 4)


,Category,Page,Target,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
_ds["Text"]=_ds["Text"].astype('string')
print(_ds.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  74682 non-null  int64 
 1   Page      74682 non-null  object
 2   Target    74682 non-null  object
 3   Text      73996 non-null  string
dtypes: int64(1), object(2), string(1)
memory usage: 2.3+ MB
None


# Exercise With a sample text

In [5]:
text_en= "i love learn more about artificial intelligence."
text = ".من عاشق یادگیری بیشتر درباره هوش مصنوعی هستم"
tokens = word_tokenize(text_en)
print(tokens)

['i', 'love', 'learn', 'more', 'about', 'artificial', 'intelligence', '.']


In [6]:
def get_wordnet_pos(tag):
    match tag[0]:
        case 'J':
            return wordnet.ADJ
        case 'V':
            return wordnet.VERB
        case 'N':
            return wordnet.NOUN
        case 'R':
            return wordnet.ADV
        case _:
            return wordnet.NOUN

tagged_words = nltk.pos_tag(tokens)
for i in range(len(tagged_words)):
    word, tag = tagged_words[i]
    tagged_words[i] = (word, get_wordnet_pos(tag))

# tagged_words = np.array(tagged_words)
print(tagged_words)

[('i', 'n'), ('love', 'v'), ('learn', 'r'), ('more', 'a'), ('about', 'n'), ('artificial', 'a'), ('intelligence', 'n'), ('.', 'n')]


In [7]:
# remove stopwords
english_stopwords = set(stopwords.words('english'))
filtered_words = [(word,tag) for word,tag in tagged_words if word.lower() not in english_stopwords]
filtered_words = [(word,tag) for word,tag in filtered_words if word.isalpha()]
print(filtered_words)

[('love', 'v'), ('learn', 'r'), ('artificial', 'a'), ('intelligence', 'n')]


In [8]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word, pos=tag) for word,tag in filtered_words]
print(lemmatized_words)

['love', 'learn', 'artificial', 'intelligence']


# Let's build project

In [9]:
import keras
import tensorflow as tf
from keras.layers import Embedding, Dense, Dropout, BatchNormalization, Input, TextVectorization
from keras.models import Model

In [10]:
dataset = _ds[['Text','Target']]
dataset=dataset.dropna()
print(dataset["Target"].value_counts())

Target
Negative      22358
Positive      20655
Neutral       18108
Irrelevant    12875
Name: count, dtype: int64


In [11]:
from sklearn.preprocessing import LabelEncoder

dataset = dataset[dataset["Target"]!="Irrelevant"]
label_encoder = LabelEncoder().fit(dataset["Target"])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
dataset["Target"] = label_encoder.transform(dataset["Target"])

print(label_mapping)

{'Negative': np.int64(0), 'Neutral': np.int64(1), 'Positive': np.int64(2)}


In [12]:
vectorize_layer = TextVectorization(
    max_tokens=30000,
    output_mode='int',
    output_sequence_length=150
)
# learning vocabulary
vectorize_layer.adapt(dataset["Text"].values)
print(f"Vocabulary Size: {vectorize_layer.vocabulary_size()}")

I0000 00:00:1758642184.380515   95158 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1105 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


Vocabulary Size: 30000


In [13]:
input_layer = Input(shape=(), dtype=tf.string)
vectorizer_layer=vectorize_layer(input_layer)
# output_dim is a hyperparameter that when grate more model will be more complex
embbed_layer = Embedding(input_dim=vectorize_layer.vocabulary_size(), output_dim=150)(vectorizer_layer)

## CNN & LSTM Channel

In [14]:
from keras.layers import Conv1D, GlobalMaxPooling1D
cnn_path = Conv1D(filters=150, kernel_size=3, activation='relu')(embbed_layer)
cnn_path = GlobalMaxPooling1D()(cnn_path)

In [15]:
from keras.layers import LSTM
lstm_path = LSTM(150)(embbed_layer)
lstm_path = Dropout(0.4)(lstm_path)

In [16]:
from keras.layers import Concatenate
merged_path = Concatenate()([cnn_path, lstm_path])

In [17]:
dense_path = Dense(128, activation='elu')(merged_path)
batchnorm_layer = BatchNormalization()(dense_path)
# final output layer
output_layer = Dense(3, activation='softmax')(batchnorm_layer)
# building final layer
model = Model(inputs=input_layer, outputs=output_layer)

## Compile Model

In [18]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization  │ (None, 150)       │          0 │ input_layer[0][0] │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 150, 150)  │  4,500,000 │ text_vectorizati… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 148, 150)  │     67,650 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 150)       │    180,600 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 150)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 150)       │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 300)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     38,528 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        387 │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,787,677 (18.26 MB)

 Trainable params: 4,787,421 (18.26 MB)

 Non-trainable params: 256 (1.00 KB)

### Train/Test Split

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset["Text"].astype(str).values, dataset["Target"].values, test_size=0.15, random_state=42)

# xtrain = tf.constant(x_train)
# xtest = tf.constant(x_test)

print(f"Train Size: {x_train.shape[0]}")
print(f"Test Size: {x_test.shape[0]}")

Train Size: 51952
Test Size: 9169


In [20]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    ModelCheckpoint(monitor="val_loss", save_best_only=True, filepath="Twittered-Model.keras")
]

model.fit(x_train, y_train, epochs=100, batch_size=150, validation_split=0.2, callbacks=callbacks)

Epoch 1/100


2025-09-23 19:13:07.229983: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300


278/278 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.7645 - loss: 0.5570 - val_accuracy: 0.8891 - val_loss: 0.7266 - learning_rate: 0.0010
Epoch 2/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9604 - loss: 0.1279 - val_accuracy: 0.8999 - val_loss: 0.2816 - learning_rate: 0.0010
Epoch 3/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9759 - loss: 0.0784 - val_accuracy: 0.9125 - val_loss: 0.2603 - learning_rate: 0.0010
Epoch 4/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9771 - loss: 0.0779 - val_accuracy: 0.9147 - val_loss: 0.2343 - learning_rate: 0.0010
Epoch 5/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9795 - loss: 0.0623 - val_accuracy: 0.9126 - val_loss: 0.2475 - learning_rate: 0.0010
Epoch 6/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9794 - loss: 0.0641 - val_accuracy: 0.8968 - val_loss: 0.2856 - learning_rate: 0.0010
Epoch 7/100
278/278 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9795 - loss: 0.0565 

In [21]:
model.evaluate(x_test, y_test)

287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9255 - loss: 0.2032


[0.20318128168582916, 0.9255098700523376]

In [22]:
reversed_mapping = dict(zip(label_mapping.values(), label_mapping.keys()))
prediction = model.predict(tf.constant(["die man! you are the worst!"]))
print(reversed_mapping[prediction.argmax()])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Negative
